In [31]:
import sys
sys.path.insert(0, '../../python')
sys.path.insert(0, '/usr/local/lib/python2.7/site-packages')
import caffe
caffe.set_mode_gpu()
net = caffe.Net( 'DeepModel.prototxt',
                'models/_iter_50000.caffemodel',
                caffe.TEST)
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
import pylab
from mpl_toolkits.mplot3d import Axes3D
import random
from matplotlib.pyplot import *
from mpl_toolkits.mplot3d import *
from numpy import *
from matplotlib import pyplot
import datetime
import cv2
for k, v in net.blobs.items():
    print k, v.data.shape

depth (64, 1, 128, 128)
joint_xyz (64, 93)
convL1 (64, 8, 128, 128)
poolL1 (64, 8, 64, 64)
convL2 (64, 16, 64, 64)
poolL2 (64, 16, 32, 32)
convL3 (64, 32, 32, 32)
poolL3 (64, 32, 16, 16)
convL4 (64, 64, 16, 16)
poolL4 (64, 64, 8, 8)
FC1 (64, 1024)
FC2 (64, 1024)
FC3 (64, 1024)
libdof (64, 55)
libxyz (64, 93)
libxyzloss ()


In [32]:
NYU_joint = [0, 3, 5, 8, 10, 13, 15, 18, 24, 25, 26, 28, 29, 30]
J = len(NYU_joint)
test_num = 8252
batch_size = (net.blobs['depth'].shape)[0]
iters = test_num / batch_size
error_per_frame = np.zeros(iters * batch_size)
out_avg_thresh = -1
thresh = np.zeros(100)
joint_error = np.zeros(J)
vis_online = False

sum = 0
a = datetime.datetime.now()
for t in range(iters):
    net.forward()
    for i in range(0,batch_size):
        img_id = batch_size * t + i
        ratio = 150 if img_id < 2440 else 130
        
        x = np.zeros(J)
        gtx = np.zeros(J)
        y = np.zeros(J)
        gty = np.zeros(J)
        z = np.zeros(J)
        gtz = np.zeros(J)
        for j in range(0,J):
            x[j]=net.blobs['libxyz'].data[i][NYU_joint[j] * 3]
            gtx[j]=net.blobs['joint_xyz'].data[i][NYU_joint[j] * 3]
            y[j]=net.blobs['libxyz'].data[i][NYU_joint[j] * 3 + 1]
            gty[j]=net.blobs['joint_xyz'].data[i][NYU_joint[j] * 3 + 1]            
            z[j]=net.blobs['libxyz'].data[i][NYU_joint[j] * 3 + 2]
            gtz[j]=net.blobs['joint_xyz'].data[i][NYU_joint[j] * 3 + 2]
            joint_error[j] += ratio * ((x[j] - gtx[j]) * (x[j] - gtx[j]) + (y[j] - gty[j]) * (y[j] - gty[j]) + (z[j] - gtz[j]) * (z[j] - gtz[j])) ** 0.5
        error_max = ((((x - gtx) * (x - gtx) + (y - gty) * (y - gty) + (z - gtz) * (z - gtz)) ** 0.5).max()) * ratio
        error_avg = ((((x - gtx) * (x - gtx) + (y - gty) * (y - gty) + (z - gtz) * (z - gtz)) ** 0.5).sum() / J) * ratio
        for k in range(int(error_max), 100):
            thresh[k] += 1
        sum += error_avg
        error_per_frame[img_id] = error_avg
        if error_avg > out_avg_thresh:
            img = (net.blobs['depth'].data[i][0] + 1) / 2 * 255
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            for i in range(J):
                cv2.circle(img, (int((x[i] + 1) / 2 * 128), int((- y[i] + 1) / 2 * 128)), 2, (255, 0, 0), 2)
                #cv2.circle(img, (int((gtx[i] + 1) / 2 * 128), int((-gty[i] + 1) / 2 * 128)), 2, (0, 0, 255), 2)
                cv2.imwrite('output/{:d}_{:.4f}.png'.format(img_id, error_avg), img)
                #cv2.imwrite('output/{:d}.png'.format(I * BATCHSIZE + see), img)
        if vis_online:
            fig=plt.figure()
            ax=fig.add_subplot((111),projection='3d')
            ax.set_xlabel('z')
            ax.set_ylabel('x')
            ax.set_zlabel('y')
            ax.scatter(z,x,y)
            ax.scatter(gtz, gtx, gty, c = 'r')
            plt.show()
    print 'iter = ', t, ', current_error = ', sum / (batch_size * (t + 1))


joint_error /= (1.0 * batch_size * iters)    
b = datetime.datetime.now()
c = b - a
tm = 1000. * c.seconds / (batch_size * iters)
print 'average time = ',tm , 'ms ', 'fps = ', 1000. / tm 
print 'average error =',  sum / (batch_size * iters)

iter =  0 , current_error =  9.73998874521
iter =  1 , current_error =  12.7097500595
iter =  2 , current_error =  14.7454223996
iter =  3 , current_error =  15.5431128955
iter =  4 , current_error =  14.8035735346
iter =  5 , current_error =  14.3862152951
iter =  6 , current_error =  15.5341940032
iter =  7 , current_error =  15.1471875759
iter =  8 , current_error =  16.5097646003
iter =  9 , current_error =  17.6269480985
iter =  10 , current_error =  17.5050741199
iter =  11 , current_error =  17.1512545509
iter =  12 , current_error =  17.0101513096
iter =  13 , current_error =  17.3937175743
iter =  14 , current_error =  17.595927391
iter =  15 , current_error =  17.6553575111
iter =  16 , current_error =  17.8118807401
iter =  17 , current_error =  17.6760052583
iter =  18 , current_error =  17.6854806854
iter =  19 , current_error =  17.8624966949
iter =  20 , current_error =  17.8839285294
iter =  21 , current_error =  18.5433771466
iter =  22 , current_error =  18.4220868215

In [37]:
fig_thresh = plt.figure()
ax_thresh = fig_thresh.add_subplot(111)
ax_thresh.plot(thresh / (1.0 * batch_size * iters) ,label='test_all', c='b')
ax_thresh.grid()
fig_thresh.savefig('max_error_under_threshold.png')

fig_error_per_frame = plt.figure()
ax_error_per_frame = fig_error_per_frame.add_subplot(111)
ax_error_per_frame.plot(error_per_frame, label='test_all', c='b')
ax_error_per_frame.grid()
fig_error_per_frame.savefig('error_per_frame.png')

NYU_joint_name = ['P1', 'P2', 'R1', 'R2', 'M1', 'M2', 'I1', 'I2', 'C', 'W1', 'W2', 'T1', 'T2', 'T3']
inds = np.arange(len(joint_error))
fig_joint_error, ax_joint_error = plt.subplots()
ax_joint_error.bar(inds, joint_error)
ax_joint_error.set_xticks(inds + 0.5)
ax_joint_error.set_xticklabels(NYU_joint_name)
plt.title("Mean Error Per Joints")
plt.ylabel("Error(mm)")
fig_joint_error.savefig('mean_error_per_joint.png')

#plt.show()

In [36]:
print thresh
fig_thresh = plt.figure()
ax_thresh = fig_thresh.add_subplot(111)
ax_thresh.plot(thresh / (1.0 * batch_size * iters) ,label='test_all', c='b')
ax_thresh.grid()
fig_thresh.savefig('max_error_under_threshold.png')
plt.show()
#plot_url = py.plot_mpl(fig, filename='mpl-basic-histogram')

[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.00000000e+00   3.00000000e+00   7.00000000e+00   2.20000000e+01
   3.80000000e+01   5.80000000e+01   9.20000000e+01   1.33000000e+02
   1.85000000e+02   2.35000000e+02   2.93000000e+02   3.41000000e+02
   4.04000000e+02   4.65000000e+02   5.46000000e+02   6.30000000e+02
   7.00000000e+02   7.87000000e+02   8.76000000e+02   9.81000000e+02
   1.09200000e+03   1.20800000e+03   1.31900000e+03   1.43200000e+03
   1.53200000e+03   1.63800000e+03   1.77100000e+03   1.89000000e+03
   2.00800000e+03   2.11600000e+03   2.23000000e+03   2.35100000e+03
   2.47900000e+03   2.60200000e+03   2.73600000e+03   2.85200000e+03
   3.00000000e+03   3.13900000e+03   3.27500000e+03   3.41100000e+03
   3.56000000e+03   3.70100000e+03   3.84900000e+03   4.00700000e+03
   4.14300000e+03   4.29900000e+03   4.44500000e+03   4.55700000e+03
   4.70000000e+03   4.82900000e+03

In [7]:
import random
import matplotlib.pyplot as plt
x=[x/1000 for x in random.sample(range(100000),100)]
xbins=range(0,len(x))
plt.hist(x, bins=xbins, color='blue')
plt.show()